# Tribunal Superior do Trabalho - TST <a class="tocSkip">

<font size=4>Danilo Paula de Souza</font> <br>
Ph.D candidate in Economics | University of Sao Paulo <br>
[https://sites.google.com/view/dpsouza](https://sites.google.com/view/dpsouza) | [danilo.paulasouza@hotmail.com](mailto:danilo.paulasouza@hotmail.com)<br><br>

Version: August 10th, 2018

***

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Packages-and-functions" data-toc-modified-id="Packages-and-functions-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Packages and functions</a></span></li></ul></div>

## Packages and functions

In [1]:
print("Hello, World!")

Hello, World!
